### Function One: make_base_url
#### Builds a url based on sensor name, table type (Full, Binned, Binned_Geo), and enabled status (True or False).

In [1]:
import requests
import json
import urllib.parse
from datetime import datetime, timedelta  # For help with managing ISO dates


# Supress certificate warnings.
from urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

def make_base_url(api_url, sensor_name, parameter_short_name, enabled_status, table_type):
    
    # Step 1: Metadata URLs can be made by adding the sensor and parameter endpoints to the base url.
    sensor_url = api_url + "/sensor"
    parameter_url = api_url + "/parameter"

    # Step 2: Find an enabled TSG sensor that we can query for a current temperature value.
    # A full list of vessel sensor is available at the sensor_url.  This list can be filtered on any filed.
    # Here we will filter on sensor_name (we know we are looking for a TSG) and on the enabled field (we want an active sensor).
    query_url = sensor_url + "/?sensor_name="+sensor_name+"&enabled="+enabled_status

    # Make the query to the REST API
    response = requests.get(query_url, verify=False)

    # Load the response as json data
    responseJSON = json.loads(response.text)

    # Grab the first sensor in the list.
    for sensor in responseJSON:
        # Print the sensor id for each record
        sensor_id = responseJSON[0]["sensor_id"]

    # Step 3. Now that we know the sensor_id for an online/active Thermosalinograph, we want to know where to look to find the data.
    # We'll need to find 4 additional peices of information before we can make a data request.  We need to know what table the data is stored in 
    # (data_table), and we need to know what field in that table the data is in (data_fieldname). We want to know the field that the data flags are in,
    # We also want the units for the parameter.

    parameter_query_url = parameter_url+"/?sensor_id="+sensor_id+"&short_name="+parameter_short_name+"&format=json"
    response = requests.get(parameter_query_url, verify=False)

    # Load the response as json data
    responseJSON = json.loads(response.text)

    if table_type == "Full":
        # Create variables to store table and field information
        data_table = responseJSON[0]['data_table']
        data_field = responseJSON[0]['data_fieldname']
        data_url = api_url + "/" + data_table

    elif table_type == "Binned_Default":
        # Create variables to store table and field information
        data_table = responseJSON[0]['binned_default_table']
        data_field = responseJSON[0]['binned_default_fieldname']
        data_url = api_url + "/" + data_table[:-8] + "_map"

    elif table_type == "Binned_Custom":
        # Create variables to store table and field information
        data_table = responseJSON[0]['binned_custom_table']
        data_field = responseJSON[0]['binned_custom_fieldname']
        data_url = api_url + "/" + data_table[:-8] + "_map"

    else:
        pass
    
    print("Thermosalinograph data can be accessed here: " + data_url)
    print("Thermosalinograph temperature data is stored in field: " + data_field)
    
    return (data_url)

In [2]:
data_url = make_base_url("https://taani-pub-ship.taani.oregonstate.edu/api", "Thermosalinograph", "Temperature", "true", "Binned_Default")

Thermosalinograph data can be accessed here: https://taani-pub-ship.taani.oregonstate.edu/api/binned_default_flow_map
Thermosalinograph temperature data is stored in field: parameter_32


In [3]:
# All datetimes are UTC, so get the current UTC time and take off one second
time_now = datetime.utcnow()
query_time = time_now - timedelta(hours=1)

# % encode the space between date and time in the query_time
query_time=urllib.parse.quote(str(query_time), safe='')

query_url = data_url+'?date_after='+str(query_time)+'&format=json'
response = requests.get(query_url, verify=False)

# Load the response as json data
responseJSON = json.loads(response.text)

print(query_url)


https://taani-pub-ship.taani.oregonstate.edu/api/binned_default_flow_map?date_after=2024-09-06%2021%3A15%3A14.659322&format=json
